# Задание

Написать эвристический алгоритм для поиска максимального независимого взвешенного (по вершинам) множества в графе.

Так как "независимое множество графа G = клика в дополнении графа G", то задачу можно переформулировать в поиск максимальной взвешенной клики в дополнении графа.

# Настройки/Гиперпараметры/Импорты

In [1]:
import numpy as np # для быстрой работы с массивами
import pandas as pd # для вывода таблицы (v 2.1.1)
import random # для рандомизированного алгоритма
import time # для подсчёта времени работы
import csv # для сохранения ответов
import math # для округления чисел в большую сторону (ceil)

In [2]:
runs = 10 # число запусков для усреднения времени
iterations = 1000 # сколько итераций должно пройти без улучшения ответа, чтобы алгоритм вернул текущий лучший

eps = 0.0001 # с какой погрешностью считать, что числа одинаковые

impact_degree = 10 # степень влиянися количества соседей у вершины на вероятность её выбора в клику
impact_weight = 1 # степень влиянися веса вершины на вероятность её выбора в клику

data_path = "./data/" # путь до папки с входными данными
solutions_path = "./solutions/" # путь, куда будут сохраняться посчитанные решения

files = ["brock200_1", "brock200_2", "brock200_3", "brock200_4", "c-fat200-1", "c-fat200-2", "c-fat200-5", "c-fat500-1", "c-fat500-10", "c-fat500-2", "c-fat500-5", "C125.9", "gen200_p0.9_44", "gen200_p0.9_55",  "johnson8-2-4",  "johnson8-4-4", "johnson16-2-4", "hamming6-2", "hamming6-4", "hamming8-2", "hamming8-4", "keller4", "MANN_a9", "MANN_a27", "MANN_a45", "p_hat300-1", "p_hat300-2", "p_hat300-3", "san200_0.7_1", "san200_0.7_2", "san200_0.9_1", "san200_0.9_2", "san200_0.9_3", "sanr200_0.7"] # все файлы, на которых должен быть протестирован код

## Вспомогательные функции, не участвующие в работе алгоритма

In [3]:
def get_complement_edges(edges: dict) -> dict:
    """
    Функция, возвращающая список смежности дополнения графа.\n
    Parameters:
        * edges: словарь смежных вершинам вершин, описывающий граф (по i-му индексу находится set со смежными i-ой вершине вершинами)\n
    Returns:
        * dict: словарь смежных вершинам вершин, описывающий дополнение графа
    """
    num_vertices = len(edges.keys()) # число вершин в графе
    all_vertices = set(range(num_vertices)) # set из всех вершин

    edges_complement = {} # словарь как список смежности дополнения графа
    for v in edges.keys(): # идём по вершинам-ключам в списке смежности для изначального графа
        edges_complement[v] = all_vertices - edges[v] - set([v]) # оставляем только те вершины, которые ранее не были смежны с вершиной v (саму вершину v тоже убираем)
    return edges_complement # возвращаем словарь смежных вершинам вершин, описывающий дополнение графа


def check_solution(edges_complement: dict, weights: list, solution, eps=0.0001) -> None:
    """
    Функция для проверка полученного лучшего решения.\n
    Parameters:
        * edges_complement: словарь смежных вершинам вершин, описывающий дополнение графа
        * weights: список весов вершин, где index — номер вершины
        * solution: решение в формате [вес независимого множества, [вершины независимого множества]]
        * eps: с какой погрешностью считать, что числа одинаковые\n
    Returns:
        * None: выкидывает исключение, если в решении есть ошибки
    """
    check_weight_sum = 0 # проверочная сумма весов вершин
    for v in solution[1]: # идём по номерам вершин независимого множества
        check_weight_sum += weights[v] # увеличиваем проверочную сумму весов вершин на значение веса рассматриваемой вершины

    if not (check_weight_sum - eps <= solution[0] <= check_weight_sum + eps): # проверка соответствия веса, полученного алгоритмом
        raise Exception("Weight is incorrect!") # выкидываем ошибку, если веса не сошлись

    for i in range(len(solution[1])-1): # идём по вершинам и проверяем, смежны ли она со всеми остальными вершинами в найденной клике ДОПОЛНЕНИЯ графа
        for j in range(i+1, len(solution[1])): # идём по последующим вершинам в клике
            if solution[1][j] not in edges_complement[solution[1][i]]: # проверяем наличие ребра между вершинами
                raise Exception(f"Nodes {i} and {j} connected!") # выкидываем ошибку, если вершины в изначальном графе были смежными (в дополнении между ними не должно быть ребра)


def transform_solution(solution, round_numbers=2) -> list:
    """
    Функция для преобразования ответа, чтобы номера вершин шли не с 0, а с 1 и по порядку.\n
    Parameters:
        * solution: решение в формате [вес независимого множества, [вершины независимого множества]]
        * round_numbers: число сохраняемых чисел после запятой\n
    Returns:
        * list: решение в формате [вес независимого множества, [отсортированные инкрементированные вершины независимого множества]]
    """
    for i in range(len(solution[1])): # идём по числу вершин в решении
        solution[1][i] += 1 # инкрементируем номер вершины (чтобы они шли не с 0, а с 1)
    return [round(solution[0], round_numbers), sorted(solution[1])] # возвращаем решение, попутно отсортировав инкрементированные вершины и округляя до round_numbers чисел после запятой


def save_solution(dataset, solution: dict) -> None:
    """
    Функция для сохранения лучших ответов.\n
    Parameters:
        * dataset: название тест-кейса
        * solution: словарь для тест-кейса с решениями в формате {"time": время на подсчёт, "weight": вес получившегося независимого множества, "independent_set": [вершины, входящие в независимое множество]}\n
    Returns:
        * None: сохраняет решение
    """
    with open(f"{solutions_path}{dataset}.csv", 'w', newline='') as file: # открываем файл для чистой записи
        writer = csv.writer(file) # создаём объект для записи
        writer.writerow([solution["weight"]]) # сохраняем размер клики (writerow — сохранение одного элемента в строку)
        writer.writerows([solution["independent_set"]]) # сохраняем вершины клики (writerows — сохранение итерационных данных по типу списка в строку)
        writer.writerow([solution["time"]]) # сохраняем время работы  (writerow — сохранение одного элемента в строку)


def show_results(solutions: dict) -> None: # solutions - словарь всех полученных ответов
    """
    Функция для вывода таблицы результатов.\n
    Parameters:
        * solutions: словарь решений в формате {"название датасета": {"time": время на подсчёт, "weight": вес получившегося независимого множества, "independent_set": [вершины, входящие в независимое множество]}, ...}\n
    Returns:
        * None: строит таблицу с полученными ответами
    """
    table = pd.DataFrame(data = [], columns=["Instance", "Time, sec", "Weight", "Independent vertices"]) # создаём pandas таблицу
    for dataset in solutions.keys(): # идём по тест-кейсам
        testcase = pd.DataFrame(data = [[dataset, solutions[dataset]["time"], solutions[dataset]["weight"], solutions[dataset]["independent_set"]]], columns=["Instance", "Time, sec", "Weight", "Independent vertices"]) # создаём "новую" таблицу для тест-кейса
        table = pd.concat([table, testcase], ignore_index=True) # объединяем таблицы
    display(table.style.hide()) # скрываем отображение индексов строк таблицы

## Считывание данных

In [4]:
data = {} 
# data - словарь вида 
# {"название датасета" : 
#     {"vertex_num": число вершин, 
#      "edge_num": число рёбер, 
#      "edges": 
#         {словарь вида вершина - set смежных ей вершин}
#     }
#  ...
# }

In [5]:
for file in files:
    data[file] = {"vertex_num": None, "edge_num": None, "edges": {}}
    with open(f"{data_path}{file}.clq", "r") as f: # открываем файл для чтения
        for row in f: # проходим по строкам
            if row[0] == "c": # если строка начинается с буквы "c" - это комментарий, пропускае строку
                continue
            elif row[0] == "p": # если строка начинается с буквы "p" - это описание проблемы, берём из этой строки число вершин и рёбер (последние два числа)
                data[file]["vertex_num"], data[file]["edge_num"] = int(row.split()[-2]), int(row.split()[-1])
            elif row[0] == "e": # если строка начинается с буквы "p" - это вершины, между которыми есть ребро
                v1, v2 = int(row.split()[-2]) - 1, int(row.split()[-1]) - 1 # запоминаем вершины (-1, чтобы не было мороки с индексацией)

                # добавляем связь вершины v1 с v2
                if v1 not in data[file]["edges"].keys(): # если это первое упоминание вершины v1 - создадим для неё set с указанием v2
                    data[file]["edges"][v1] = {v2}
                elif v2 not in data[file]["edges"][v1]: # иначе - просто добавим v2 в set смежных вершин v1
                    data[file]["edges"][v1].add(v2)

                # аналогично, но относительно вершины v2
                if v2 not in data[file]["edges"].keys():
                    data[file]["edges"][v2] = {v1}
                elif v1 not in data[file]["edges"][v2]:
                    data[file]["edges"][v2].add(v1)
        data[file]["edges"] = dict(sorted(data[file]["edges"].items())) # отсортируем вершины в словаре (в set для ключа словаря вершины уже отсортированы)

In [6]:
data["johnson8-2-4"] # пример данных

{'vertex_num': 28,
 'edge_num': 210,
 'edges': {0: {5, 8, 9, 12, 13, 14, 17, 18, 19, 20, 23, 24, 25, 26, 27},
  1: {4, 7, 9, 11, 13, 14, 16, 18, 19, 20, 22, 24, 25, 26, 27},
  2: {3, 6, 9, 10, 13, 14, 15, 18, 19, 20, 21, 24, 25, 26, 27},
  3: {2, 7, 8, 11, 12, 14, 16, 17, 19, 20, 22, 23, 25, 26, 27},
  4: {1, 6, 8, 10, 12, 14, 15, 17, 19, 20, 21, 23, 25, 26, 27},
  5: {0, 6, 7, 10, 11, 14, 15, 16, 19, 20, 21, 22, 25, 26, 27},
  6: {2, 4, 5, 11, 12, 13, 16, 17, 18, 20, 22, 23, 24, 26, 27},
  7: {1, 3, 5, 10, 12, 13, 15, 17, 18, 20, 21, 23, 24, 26, 27},
  8: {0, 3, 4, 10, 11, 13, 15, 16, 18, 20, 21, 22, 24, 26, 27},
  9: {0, 1, 2, 10, 11, 12, 15, 16, 17, 20, 21, 22, 23, 26, 27},
  10: {2, 4, 5, 7, 8, 9, 16, 17, 18, 19, 22, 23, 24, 25, 27},
  11: {1, 3, 5, 6, 8, 9, 15, 17, 18, 19, 21, 23, 24, 25, 27},
  12: {0, 3, 4, 6, 7, 9, 15, 16, 18, 19, 21, 22, 24, 25, 27},
  13: {0, 1, 2, 6, 7, 8, 15, 16, 17, 19, 21, 22, 23, 25, 27},
  14: {0, 1, 2, 3, 4, 5, 15, 16, 17, 18, 21, 22, 23, 24, 27},
  15

In [7]:
weights = {}
# data - словарь вида 
# {"название датасета" : [веса вершин в графе], ...}

In [8]:
for dataset in data.keys(): # идём по тест-кейсам
    weights[dataset] = [math.ceil(10*i / data[dataset]["vertex_num"]) * 0.1 for i in range(1, data[dataset]["vertex_num"]+1)] 
# задаём веса вершин по формуле w_i = ceil(10*i / n) * 0.1, где n - число вершин, i - номер вершины (начиная с 1)
# То есть первые 10% вершин имеют вес 0.1, следующие 10% - вес 0.2, ..., последние 10% - вес 1.0.

In [9]:
weights["johnson8-2-4"] # пример весов вершин

[0.1,
 0.1,
 0.2,
 0.2,
 0.2,
 0.30000000000000004,
 0.30000000000000004,
 0.30000000000000004,
 0.4,
 0.4,
 0.4,
 0.5,
 0.5,
 0.5,
 0.6000000000000001,
 0.6000000000000001,
 0.7000000000000001,
 0.7000000000000001,
 0.7000000000000001,
 0.8,
 0.8,
 0.8,
 0.9,
 0.9,
 0.9,
 1.0,
 1.0,
 1.0]

# Реализация алгоритма

In [10]:
def randomized_max_weighted_clique(edges: dict, weights: list, impact_degree, impact_weight, iterations: int=10) -> list:
    """
    Функция для получения рандомизированного решения задачи о максимальной взвешенной клике.\n
    Parameters:
        * edges: словарь смежных вершинам вершин
        * weights: список с весами вершин
        * impact_degree: степень влиянися количества соседей у вершины на вероятность её выбора в клику
        * impact_weight: степень влиянися веса вершины на вероятность её выбора в клику
        * iterations: через сколько попыток без улучшения решения выходить из алгоритма\n
    Returns:
        * list: данные о найденной взвешенной клике в формате [вес, [вершина в клике 1, ..., вершина в клике k]]
    """
    num_vertices = len(weights) # == len(edges.keys()), число вершин в графе
    original_candidates = set(edges.keys()) # set вершин (изначально все являются кандидатами в клику)
    original_candidates_degrees = [len(edges[v]) for v in original_candidates] # создаём список степеней вершин (индекс - номер вершины, так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)

    probability_weights = [impact_degree*original_candidates_degrees[v] + impact_weight*weights[v]  for v in edges.keys()] # создаём список с вероятностями выбора вершины в клику в зависимости от (число смежных вершин)*(коэффициент степени вершины) + (вес вершины)*(коэффициент веса вершины)

    best_weighted_clique = [] # текущая лучшая взвешенная клика
    best_weight = 0 # вес лучшей взвешенной клики

    attempts = 0 # текущее число попыток без улучшения результата
    while attempts < iterations: # запускаем алгоритм, пока число попыток без изменения результата не превысит счётчик iterations
        weighted_clique = [] # создаём "пустую" клику
        weight = 0 # вес клики на текущей попытке

        candidates = original_candidates.copy() # копируем set всех кандидатов
        while len(candidates) != 0: # пока есть кандидаты — пытаемся добавить их в клику в зависимости от их probability_weights
            candidates_probability_weights = [probability_weights[i] for i in candidates] # обновляем вероятности попадания кандидатов в клику (оставляем вероятности только допустимых вершин) для итерациии случайного выбора
            
            v = random.choices(population=list(candidates), weights=candidates_probability_weights, k=1)[0] # случайным образом выбираем вершину в клику в соответствии с её вероятнотью (чем больше степень и вес относительно других вершин — тем выше вероятность) (переводим candidates в список для случайного выбора)
            weighted_clique.append(v) # добавляем выбранную вершину в клику
            weight += weights[v] # увеличиваем вес рассматриваемой клики

            candidates = candidates.intersection(edges[v]) # среди кандидитов оставляем только тех, кто смежен со всеми вершинами в текущей клике (итеративно этот список постоянно уменьшается с добавлением новых вершин в клику)

        if weight > best_weight: # если нашли новую лучшую взвешенную клику, то запоминаем её
            best_weighted_clique = weighted_clique.copy() # сохраняем содержимое лучшей взвешенной клики
            best_weight = weight # обновляем лучший вес
            # print(f"attempt {attempts} with new best: {best_weighted_clique}")
            attempts = 0 # обнуляем число итераций без улучшения решения
        else:
            attempts += 1 # увеличиваем число итераций без улучшения решения

    return [best_weight, best_weighted_clique] # возвращаем вес лучшей взвешенной клики и её содержимое

#### Тестирование

In [11]:
solutions = {} 
# словарь для ответов алгоритма
# {"название датасета" : 
#     {"time": время на подсчёт,
#      "weight": вес найденного независимого множества (клики дополнения графа),
#      "independent_set": [вершины, входящие в независимое множество],
#     }
# }

In [12]:
for dataset in data.keys(): # идём по тест-кейсам
    time_start = time.perf_counter() # замеряем время начала выполнения
    for i in range(runs): # делаем runs запусков для усреднения времени
        edges_complement = get_complement_edges(data[dataset]["edges"]) # переходим к дополнению графа (от независимого множества к клике) (в цикле для честного подсчёта времени)
        solution = randomized_max_weighted_clique(edges=edges_complement, weights=weights[dataset], impact_degree=impact_degree, impact_weight=impact_weight, iterations=iterations) # запускаем рандомизированный алгоритм
        # print(f"run: {i}, solution: {solution}")
    time_working = time.perf_counter() - time_start # считаем сколько времени работал алгоритм

    check_solution(edges_complement=edges_complement, weights=weights[dataset], solution=solution) # проверка решения (последнего полученного за runs запусков, оно может быть не лучшим)
    solution = transform_solution(solution) # сортирует вершины взвешенной клики в порядке возрастания их номера и возвращает нумерацию с единицы
    solutions[dataset] = {"time": time_working/runs, "weight": solution[0], "independent_set": solution[1]} # добавление ответа в словарь с ответами
    save_solution(dataset, solutions[dataset]) # сохранение полученного ответа
    print(f"{dataset}: {solutions[dataset]}") # вывод полученного ответа

brock200_1: {'time': 0.11831016999999981, 'weight': 4.5, 'independent_set': [91, 187, 190, 196, 198]}
brock200_2: {'time': 0.1617121900000001, 'weight': 7.4, 'independent_set': [56, 112, 152, 157, 164, 187, 188, 193, 196]}
brock200_3: {'time': 0.14140774, 'weight': 5.7, 'independent_set': [53, 127, 144, 177, 184, 188, 189]}
brock200_4: {'time': 0.12463324, 'weight': 5.3, 'independent_set': [130, 156, 164, 176, 183, 194]}
c-fat200-1: {'time': 0.5122809500000003, 'weight': 12.9, 'independent_set': [54, 67, 73, 102, 123, 130, 135, 151, 153, 157, 169, 174, 180, 182, 186, 192, 200]}
c-fat200-2: {'time': 0.2980272600000006, 'weight': 7.0, 'independent_set': [58, 86, 124, 144, 170, 172, 174, 182, 186]}
c-fat200-5: {'time': 0.10281171999999969, 'weight': 2.9, 'independent_set': [170, 195, 200]}
c-fat500-1: {'time': 2.6131518599999994, 'weight': 24.9, 'independent_set': [26, 30, 37, 137, 148, 164, 166, 212, 215, 230, 236, 260, 268, 285, 301, 303, 320, 322, 352, 363, 368, 379, 398, 423, 435, 440

In [13]:
show_results(solutions)

C:\Users\User\Documents\TEMP\ipykernel_16348\1291053311.py:83: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  table = pd.concat([table, testcase], ignore_index=True) # объединяем таблицы


Instance,"Time, sec",Weight,Independent vertices
brock200_1,0.118310,4.500000,"[91, 187, 190, 196, 198]"
brock200_2,0.161712,7.400000,"[56, 112, 152, 157, 164, 187, 188, 193, 196]"
brock200_3,0.141408,5.700000,"[53, 127, 144, 177, 184, 188, 189]"
brock200_4,0.124633,5.300000,"[130, 156, 164, 176, 183, 194]"
c-fat200-1,0.512281,12.900000,"[54, 67, 73, 102, 123, 130, 135, 151, 153, 157, 169, 174, 180, 182, 186, 192, 200]"
c-fat200-2,0.298027,7.000000,"[58, 86, 124, 144, 170, 172, 174, 182, 186]"
c-fat200-5,0.102812,2.900000,"[170, 195, 200]"
c-fat500-1,2.613152,24.900000,"[26, 30, 37, 137, 148, 164, 166, 212, 215, 230, 236, 260, 268, 285, 301, 303, 320, 322, 352, 363, 368, 379, 398, 423, 435, 440, 450, 465, 472, 474, 488, 490, 492, 494, 496, 498]"
c-fat500-10,0.368848,3.900000,"[421, 457, 471, 483]"
c-fat500-2,1.221492,14.400000,"[28, 312, 332, 343, 368, 396, 402, 417, 421, 425, 459, 470, 474, 479, 485, 490, 494]"


**Возможные улучшения:**
* Пересчёт степеней вершин кандидатов после добавления вершины в клику, чтобы кандидаты, что имели большую степень изначально — в ходе работы алгоритма пересчитывались (чтобы не учитывать "бесполезные" связи с вершинами, что не могут попасть в клику).